<img src="../../images/BikeDNA_logo.svg" width="250"  alt="BikeDNA logo" style="display:block; margin-left: auto; margin-right: auto;">
<a href="https://github.com/anerv/BikeDNA">Github</a>

# Example reference data preprocessing: GeoDanmark

This notebook provides an example of how a spatial dataset with data on cycling infrastructure can be converted to the format required by BikeDNA. When using your own data, The preprocessing must be adapted to content and format.

The data used in this notebook are from *GeoDanmark* and were downloaded from [dataforsyningen.dk](https://dataforsyningen.dk/) under the [GeoDanmark license](https://www.geodanmark.dk/wp-content/uploads/2022/08/Vilkaar-for-brug-af-frie-geografiske-data_GeoDanmark-grunddata-august-2022.pdf).

As stated in the data set requirements, the reference data should:

- only contain **cycling infrastructure** (i.e. not also the regular street network)
- have all geometries as **LineStrings** (not MultiLineString)
- for each row, the geometry should be a **straight** LineString only defined by its start- and end nodes
- have start/end nodes at **intersections**
- be in a **CRS** recognised by GeoPandas
- contain a column describing whether each feature is a physically **protected**/separated infrastructure or if it is **unprotected**
- contain a column describing whether each feture is **bidirectional** or not
- contain a column describing how features have been **digitized** ('geometry type')
- contain a column with a unique **ID** for each feature

In [ ]:
import contextily as cx
import folium
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
from shapely.ops import linemerge

from src import graph_functions as gf
from src import plotting_functions as pf

%run ../settings/tiledict.py


In [ ]:
geodk = gpd.read_parquet("../../data/REFERENCE/dk/raw/geodk_bike.parquet")
geodk.sample(10)

Our dataset contains the entire road network, including bicycle tracks and lanes. We are only interested in the dedicated cycling infrastructure and thus need to select a subset of the data.
We also only want to include infrastructure that is completed or under construction.

Some of the data might be outside of the study area we are interested in, but the data processing in notebook 01 will clip all data to the desired extent.

In [ ]:
# Creating subset only with existing cycling infrastructure

geodk_selection = geodk.loc[
    (geodk.vejkategori.isin(["Cykelsti langs vej", "Cykelbane langs vej"]))
    & (geodk.status.isin(["Anlagt", "Under anlæg"]))
].copy()

In [ ]:
geodk_selection.explore()

For all code to run without errors, our dataset can only contain LineString geometries. Let's check what we have:

In [ ]:
geodk_selection.geom_type.unique()

In this dataset, we only have MultiLineStrings. To fix this, we first try to merge the MultiLineStrings. 
If some of the MultiLinestrings are not connected (i.e. there are gaps in the lines), the aboves step will not be able to merge them. In that case we can instead 'explode' them.

In [ ]:
geodk_linestrings = geodk_selection.copy()
# Convert MultiLineStrings to LineString
geodk_linestrings["geometry"] = geodk_linestrings["geometry"].apply(
    lambda x: linemerge(x) if x.geom_type == "MultiLineString" else x
)

if (
    len(geodk_linestrings.geom_type.unique()) > 1
    or geodk_linestrings.geom_type.unique()[0] != "LineString"
):

    print("Exploding MultiLineStrings...")
    geodk_linestrings = geodk_selection.explode(ignore_index=True)

assert len(geodk_linestrings.geom_type.unique()) == 1
assert geodk_linestrings.geom_type.unique()[0] == "LineString"
geodk_linestrings.geom_type.unique()

For the code to work, the data need to be in a CRS recognized by GeoPandas, and to have that CRS defined. Let's check that we have a CRS defined:

In [ ]:
geodk_linestrings.crs

The analysis of data quality is based on the concept of a *network*. For the results to be accurate we need a dataset with nodes at intersections (i.e. where the lines defining the cycling infrastructure intersect).

Use the folium plot below to check that you do have nodes at intersections.
If not, this will have to be fixed - or it will be an aspect of low data quality that will become apparent in the analysis of data quality...

Don't worry if there are more nodes than just those at intersections and start/end points - we will take care of that in the data loading notebook.

In [ ]:
G = momepy.gdf_to_nx(
    geodk_linestrings.to_crs("EPSG:25832"), approach="primal", directed=True
)  # We reproject the network data to avoid warnings - final reprojection will happen later

nodes, edges = momepy.nx_to_gdf(G)

In [ ]:
# Feature groups 
edges_folium = pf.make_edgefeaturegroup(
    gdf=edges, mycolor="black", myweight=2, nametag="edges", show_edges=True
)

nodes_folim = pf.make_nodefeaturegroup(
    gdf=nodes, mycolor="red", mysize=2, nametag="nodes", show_nodes=True
)

feature_groups = [edges_folium, nodes_folim]

m = pf.make_foliumplot(
    feature_groups=feature_groups,
    layers_dict=folium_layers,
    center_gdf=nodes,
    center_crs=nodes.crs,
)

display(m)

We don't technically need to drop any unnecessary columns, but let's avoid loading unnecessary data later on.

In [ ]:
geodk_linestrings.columns

In [ ]:
# Drop unnecessary columns

geodk_linestrings = geodk_linestrings[["id_lokalid", "vejkategori", "geometry"]]

For consistency, we rename all column names to lower case letters:

In [ ]:
geodk_linestrings = gf.clean_col_names(geodk_linestrings)

For this dataset we assume of all features to be 'true' geometry mappings and one directional, so we can specify this in config file and do not have to add it to the data.

The rest of the pre-processing, such as projecting to the chosen CRS, clipping the data to the study area etc. will happen in [notebook 2a](../REFERENCE/2a_initialize_reference.ipynb).

**Final dataset**

In [ ]:
geodk_linestrings.sample(10)

**Export dataset**

In [ ]:
geodk_linestrings.to_parquet(
    "../../data/reference/dk/processed/reference_data.parquet"
)



*Contains data from GeoDanmark (retrieved spring 2023)*
*© SDFE (Styrelsen for Dataforsyning og Effektivisering og Danske kommuner)*

*License: [GeoDanmark](https://www.geodanmark.dk/wp-content/uploads/2022/08/Vilkaar-for-brug-af-frie-geografiske-data_GeoDanmark-grunddata-august-2022.pdf)*